# 1. Tree

In [4]:
class Tree:
    def __init__(self):
        self.node=[]
        self.depth={}
        self.edge=[]
        self.parent={}
        self.child={}
        self.root=None
    
    def add_edge(self,v1,v2):
        #an arc from v1 to v2
        if v1 not in self.node:
            self.node.append(v1)
        if v2 not in self.node:
            self.node.append(v2)
        self.edge.append((v1,v2))
    
    def construct_tree(self):
        for v in self.node:
            self.child[v]=[]
            self.parent[v]=None
        for (v1,v2) in self.edge:
            self.parent[v2]=v1
            self.child[v1].append(v2)
    
    def find_root(self):
        v=self.node[0]
        while self.parent[v]!=None:
            v=self.parent[v]
        self.root=v
    
    def find_depth(self, node, dep=0):
        self.depth[node]=dep;
        for child in self.child[node]:
            self.find_depth(child,dep+1)

# 2. Read Tree

In [7]:
def readtrees(patient):
    tree_file = open("data/input/T_%s.txt"%patient,"r")
    N_clusters=tree_file.readline().split()[1]
    N_clusters=int(N_clusters)
    N_trees=tree_file.readline().split()[0]
    N_trees=int(N_trees)
    trees=[]
    for i in range(N_trees):
        tree=Tree()
        for j in range(N_clusters):
            line=tree_file.readline()
            if j==0: continue
            v1,v2=line.rstrip("\n").split()
            tree.add_edge(v1,v2)
        tree.construct_tree()
        tree.find_root()
        tree.find_depth(tree.root)
        trees.append(tree)
        return trees
trees=readtrees("CRUK0002")